# Use Closed-Form Policy to Play GuessingGame-v0

In [1]:
import sys
import logging
import itertools

import numpy as np
np.random.seed(0)
import gym
import gym_toytext

logging.basicConfig(level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

In [2]:
env = gym.make('GuessingGame-v0')
env.seed(0)
for key in vars(env.spec):
    logging.info('%s: %s', key, vars(env.spec)[key])
for key in vars(env.unwrapped):
    logging.info('%s: %s', key, vars(env.unwrapped)[key])

00:00:00 [INFO] id: GuessingGame-v0
00:00:00 [INFO] entry_point: gym_toytext:GuessingGame
00:00:00 [INFO] reward_threshold: None
00:00:00 [INFO] nondeterministic: False
00:00:00 [INFO] max_episode_steps: 200
00:00:00 [INFO] order_enforce: True
00:00:00 [INFO] _kwargs: {}
00:00:00 [INFO] _env_name: GuessingGame
00:00:00 [INFO] range: 1000
00:00:00 [INFO] bounds: 10000
00:00:00 [INFO] action_space: Box([-10000.], [10000.], (1,), float32)
00:00:00 [INFO] observation_space: Discrete(4)
00:00:00 [INFO] number: -548.1826058139857
00:00:00 [INFO] guess_count: 0
00:00:00 [INFO] guess_max: 200
00:00:00 [INFO] observation: 0
00:00:00 [INFO] np_random: RandomState(MT19937)
00:00:00 [INFO] spec: EnvSpec(GuessingGame-v0)


In [3]:
class ClosedFormAgent:
    def __init__(self, _):
        pass

    def reset(self, mode=None):
        self.low = env.action_space.low
        self.high = env.action_space.high
        self.action = None

    def step(self, observation, _reward, _done):
        if observation == 1:
            self.low = self.action
        elif observation == 3:
            self.high = self.action
        self.action = (self.low + self.high) / 2.
        return self.action

    def close(self):
        pass


agent = ClosedFormAgent(env)

In [4]:
def play_episode(env, agent, max_episode_steps=None, mode=None, render=False):
    observation, reward, done = env.reset(), 0., False
    agent.reset(mode=mode)
    episode_reward, elapsed_steps = 0., 0
    while True:
        action = agent.step(observation, reward, done)
        if render:
            env.render()
        if done:
            break
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        elapsed_steps += 1
        if max_episode_steps and elapsed_steps >= max_episode_steps:
            break
    agent.close()
    return episode_reward, elapsed_steps


logging.info('==== test ====')
episode_rewards = []
for episode in range(100):
    episode_reward, elapsed_steps = play_episode(env, agent)
    episode_rewards.append(episode_reward)
    logging.debug('test episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
logging.info('average episode reward = %.2f ± %.2f',
        np.mean(episode_rewards), np.std(episode_rewards))

00:00:00 [INFO] ==== test ====
00:00:00 [DEBUG] test episode 0: reward = 1.00, steps = 9
00:00:00 [DEBUG] test episode 1: reward = 1.00, steps = 6
00:00:00 [DEBUG] test episode 2: reward = 1.00, steps = 9
00:00:00 [DEBUG] test episode 3: reward = 1.00, steps = 9
00:00:00 [DEBUG] test episode 4: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 5: reward = 1.00, steps = 9
00:00:01 [DEBUG] test episode 6: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 7: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 8: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 9: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 10: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 11: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 12: reward = 1.00, steps = 8
00:00:01 [DEBUG] test episode 13: reward = 1.00, steps = 10
00:00:01 [DEBUG] test episode 14: reward = 1.00, steps = 9
00:00:01 [DEBUG] test episode 15: reward = 1.00, steps = 8
00:00:01 [DEBUG] test episo

In [5]:
env.close()